In [1]:
!pip install drawsvg

In [1]:
import drawsvg as dw
from math import floor, ceil
import ipywidgets as widgets



# Instructions on how to run
Set the dimensions of the filter, your input tile, and your input matrix in the below cell. Then run all the cells.
Then go to cell 5 and cell 6 and you will be able to interact with the widgets.

Input tile is a square 2D matrix. So size is same along both dimensions.
The input matrix can be of different sizes along the two dimensions.

In [2]:

FILTER_RADIUS = 2
INPUT_TILE_DIM = 8
OUTPUT_TILE_DIM = INPUT_TILE_DIM - 2*FILTER_RADIUS
MATRIX_WIDTH = 50
MATRIX_HEIGHT = 50

In [3]:

def draw_block(width, height, blockid='matrix'):
   
    block =  dw.Group(id='bl{}'.format(blockid), fill='yellow', stroke='black')
    rec = dw.Rectangle(0, 0, 10, 10, stroke="black")
    block_items = []
    for j in range(height):
        block_items.append([])
        for i in range(width):
            r = dw.Rectangle(10*i, 10*j, 10, 10, fill='white', id='bl{}rec{}'.format(blockid, j*width+i))
            block_items[j].append(r)
            block.append(r)
    #d.append(dw.Use(rec, transform="translate(0, 10)"))
    #d.append(block)
    return block, block_items

def draw_grid(grid_width, grid_height, block_width, block_height):
    grid = dw.Group(id='gr')
    grid_items = []
    for j in range(grid_height):
        grid_items.append([])
        for i in range(grid_width):
            block, block_items = draw_block(block_width, block_height, blockid=j*grid_width+i)
            g = dw.Use(block, 0, 0, transform="translate({}, {})".format(10*block_width*i+10*i, 10*block_height*j+10*j))
            grid_items[j].append(block_items)
            grid.append(g)
    return grid, grid_items




d = dw.Drawing(2000, 2000)
# draw input matrix with extra rows and columns (an INPUT_TILE_DIM worth of extra rows and columns
# at the top, bottom, left, and right of the matrix) that would be the ghost cells. 
# we probably don't need to draw these many ghost rows and columns, but this way we get a really
# big buffer that we don't have to worry about any overflows when rendering the output
matrix, matrix_items = draw_block(MATRIX_WIDTH+2*INPUT_TILE_DIM, MATRIX_HEIGHT+2*INPUT_TILE_DIM)
d.append(dw.Use(matrix, 0, 0))
# coloring ghost cells with sky blue color to distinguish them from the actual data
for j in range(MATRIX_HEIGHT+2*INPUT_TILE_DIM):
    for i in range(MATRIX_WIDTH+2*INPUT_TILE_DIM):
        if (i-INPUT_TILE_DIM) < 0 or (j - INPUT_TILE_DIM) < 0 or i >= (MATRIX_WIDTH+INPUT_TILE_DIM) or j >= (MATRIX_HEIGHT+INPUT_TILE_DIM):
            matrix_items[j][i].args['fill'] = '#b2ffff'

# draw cuda blocks
grid_width = ceil(MATRIX_WIDTH/OUTPUT_TILE_DIM)
grid_height = ceil(MATRIX_WIDTH/OUTPUT_TILE_DIM)
block_width = INPUT_TILE_DIM
block_height = INPUT_TILE_DIM
grid, grid_items = draw_grid(grid_width, grid_height, block_width, block_height)
d.append(dw.Use(grid, 0, (MATRIX_HEIGHT+2*INPUT_TILE_DIM)*10+10))




In [4]:


def reset_grid_colors(grid_items):
    for j in range(grid_height):
        for i in range(grid_width):
            reset_block_colors(grid_items[j][i])

def reset_block_colors(block_items):
    for j in range(block_height):
        for i in range(block_width):
            block_items[j][i].args['fill'] = 'white'

def reset_matrix_colors(matrix_items):
    for j in range(MATRIX_HEIGHT+2*INPUT_TILE_DIM):
        for i in range(MATRIX_WIDTH+2*INPUT_TILE_DIM):
            if (i-INPUT_TILE_DIM) < 0 or (j - INPUT_TILE_DIM) < 0 or i >= (MATRIX_WIDTH+INPUT_TILE_DIM) or j >= (MATRIX_HEIGHT+INPUT_TILE_DIM):
                matrix_items[j][i].args['fill'] = '#b2ffff'
            else:
                matrix_items[j][i].args['fill'] = 'white'



In the below widget, you will see a representation of your input matrix (with blue ghost cells) and then the grid of thread blocks that is created. Each thread block will load one input tile. g_row and g_col represent the row and column of the thread block in the grid of thread blocks. The selected block is highlighted in yellow. And in the input matrix you will see the elements highlighted in yellow, which represent the input tile loaded by that block.

In [5]:
@widgets.interact
def show_input_tile(g_row=widgets.BoundedIntText(min=0,max=grid_height-1), g_col=widgets.BoundedIntText(min=0, max=grid_width-1)):
    """
    highlight input tile selection and 
    highlight cuda block that fills the input tile
    """
    reset_grid_colors(grid_items)
    reset_matrix_colors(matrix_items)
    block = grid_items[g_row][g_col]

    for b_row in range(block_height):
        for b_col in range(block_width):
            # m_row and m_col identify the coordinates of the element in the matrix
            # that is loaded into the input tile by the thread at (b_col,b_row) in the 
            # selected block. 
            # The g_col*OUTPUT_TILE_DIM, g_row*OUTPUT_TILE_DIM gives the starting point of the 
            # output tile that is calculated by the block. 
            # b_col-FILTER_RADIUS,b_row-FILTER_RADIUS gives the offset from the starting
            # point at which the thread b_col,b_row loads the element into the input tile
            # the +INPUT_TILE_DIM is because we have to offset against all the extra ghost
            # cells in the visuals. 
            m_row = (g_row*OUTPUT_TILE_DIM)+b_row-FILTER_RADIUS + INPUT_TILE_DIM
            m_col = (g_col*OUTPUT_TILE_DIM)+b_col-FILTER_RADIUS + INPUT_TILE_DIM
            matrix_items[m_row][m_col].args['fill'] = 'yellow'
            block[b_row][b_col].args['fill'] = 'yellow'

    return d
        
        

interactive(children=(BoundedIntText(value=0, description='g_row', max=12), BoundedIntText(value=0, descriptio…

The below widget is similar to the above widget, with the addition of the red highlights. The red highlights in the input matrix represent those elements in the input tile for which the output is calculated. You can see the surrounding yellow elements of the input tile would not have enough input elements around them to calculate their output. In the grid of blocks, the red highlights represent those threads that calculate an output element. The other yellow threads in the block are disabled after they load their respective input elements into the input tile.

If you step through the different blocks along the rows and columns, you will see that each input neighboring input tile overlap each other by FILTER_RADIUS amount, but the output elements don't overlap. Stepping through all of them, you can see that output elements for all elements of the entire input matrix are calculated. You can also see that for some blocks, the input tile goes outside the input matrix into the ghost cells (the blue cells) which have a default value since they are not actually in the input data.

In [6]:
@widgets.interact
def show_output_tile(g_row=widgets.BoundedIntText(min=0,max=grid_height-1), g_col=widgets.BoundedIntText(min=0, max=grid_width-1)):
    """
    highlight output tile that is calculated from the input tile and 
    highlight the subset of the cuda block that calculates the output tile elements
    """
    reset_grid_colors(grid_items)
    reset_matrix_colors(matrix_items)
    block = grid_items[g_row][g_col]

    for b_row in range(block_height):
        for b_col in range(block_width):

            m_row = (g_row*OUTPUT_TILE_DIM)+b_row-FILTER_RADIUS + INPUT_TILE_DIM
            m_col = (g_col*OUTPUT_TILE_DIM)+b_col-FILTER_RADIUS + INPUT_TILE_DIM
            out_row = b_row - FILTER_RADIUS
            out_col = b_col - FILTER_RADIUS
            # checking that out_col,out_row has a value from 0 to OUTPUT_TILE_DIM-1 
            # helps 'turn off' the threads that shouldn't calculate an output element in the
            # output tile (because its corresponding element in the input tile doesn't have enough
            # surrouding elements to be multiplied with the filter).
            if out_row >= 0 and out_row < OUTPUT_TILE_DIM and out_col >= 0 and out_col < OUTPUT_TILE_DIM:
                matrix_items[m_row][m_col].args['fill'] = 'red'
                block[b_row][b_col].args['fill'] = 'red'
            else:
                matrix_items[m_row][m_col].args['fill'] = 'yellow'
                block[b_row][b_col].args['fill'] = 'yellow'
            

    return d

interactive(children=(BoundedIntText(value=0, description='g_row', max=12), BoundedIntText(value=0, descriptio…